In [6]:
import polars as pl
from sentence_transformers import SentenceTransformer, util

In [29]:
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
df = pl.read_csv("./csv_data//deoksan.csv")
col_lis = list(df.schema.keys())

In [33]:
long_df = df.unpivot(
    index=["time"], 
    variable_name="sensor_key", 
    value_name="sensor_value"
)

In [35]:
len(long_df)

54181864

In [30]:
print(len(col_lis))
print(col_lis)

12
['time', 'curr', 'currR', 'currS', 'currT', 'Ground', 'PT100', 'Vibra', 'Volt', 'VoltR', 'VoltS', 'VoltT']


In [ ]:
test_lis = ["time", "curr", "open", "high", "low", "close", "volume"]

In [ ]:
embeddings_col = model.encode(col_lis)
embeddings_test = model.encode(test_lis)

In [ ]:
cos_sim = util.pytorch_cos_sim(embeddings_col, embeddings_test)

In [ ]:
cos_sim

tensor([[1.0000, 0.2815, 0.3500, 0.3599, 0.2940, 0.3690, 0.2843],
        [0.2815, 1.0000, 0.2494, 0.2326, 0.1609, 0.2941, 0.1359],
        [0.2526, 0.8962, 0.2469, 0.1903, 0.1107, 0.2960, 0.1507],
        [0.2837, 0.7859, 0.2376, 0.1964, 0.1306, 0.2623, 0.1399],
        [0.3048, 0.8718, 0.2288, 0.1948, 0.1153, 0.3028, 0.1356],
        [0.2937, 0.1676, 0.2247, 0.3562, 0.3964, 0.2598, 0.2770],
        [0.2539, 0.1860, 0.1843, 0.2260, 0.2490, 0.2050, 0.2276],
        [0.1850, 0.2069, 0.2003, 0.1871, 0.1757, 0.2397, 0.1827],
        [0.3318, 0.2287, 0.2352, 0.3520, 0.3484, 0.3218, 0.5543],
        [0.2524, 0.2876, 0.2477, 0.3130, 0.2403, 0.2996, 0.4205],
        [0.3014, 0.2136, 0.2043, 0.3438, 0.3655, 0.2958, 0.5674],
        [0.2648, 0.2697, 0.2373, 0.3036, 0.2599, 0.3133, 0.3714]])

In [ ]:
import pandas as pd
df_sim = pd.DataFrame(
    cos_sim.numpy(),
    index=col_lis,
    columns=test_lis
)

In [ ]:
df_sim

,time,curr,open,high,low,close,volume
time,1.000000,0.281514,0.349970,0.359886,0.294038,0.369033,0.284349
curr,0.281514,1.000000,0.249423,0.232571,0.160885,0.294118,0.135923
currR,0.252592,0.896221,0.246941,0.190300,0.110683,0.296023,0.150738
currS,0.283679,0.785856,0.237553,0.196365,0.130628,0.262291,0.139895
currT,0.304782,0.871761,0.228802,0.194801,0.115335,0.302783,0.135615
Ground,0.293742,0.167581,0.224695,0.356201,0.396437,0.259835,0.276982
PT100,0.253947,0.186023,0.184275,0.226033,0.248988,0.204997,0.227554
Vibra,0.185005,0.206896,0.200322,0.187099,0.175703,0.239717,0.182725
Volt,0.331780,0.228741,0.235239,0.351962,0.348422,0.321800,0.554284
VoltR,0.252428,0.287573,0.247696,0.313022,0.240265,0.299600,0.420484


In [ ]:
results = []
for i, src in enumerate(col_lis):
    best_idx = cos_sim[i].argmax().item()
    best_score = cos_sim[i][best_idx].item()
    best_target = test_lis[best_idx]
    
    results.append({
        "Raw_Column": src,
        "Mapped_Ontology": best_target,
        "Score": round(best_score, 2),
        "Assessment": "✅ Good" if best_score > 0.5 else "⚠️ Check" # 임계값은 상황따라 조절
    })

pd.DataFrame(results)

,Raw_Column,Mapped_Ontology,Score,Assessment
0,time,time,1.00,✅ Good
1,curr,curr,1.00,✅ Good
2,currR,curr,0.90,✅ Good
3,currS,curr,0.79,✅ Good
4,currT,curr,0.87,✅ Good
5,Ground,low,0.40,⚠️ Check
6,PT100,time,0.25,⚠️ Check
7,Vibra,close,0.24,⚠️ Check
8,Volt,volume,0.55,✅ Good
9,VoltR,volume,0.42,⚠️ Check


In [ ]:
# 컬럼명만 추출해서 임베딩 후 shape와 일부 값만 출력하도록 개선
column_names = [col for col, dtype in df.schema.items()]
embeddings = model.encode(column_names)
print("Embeddings shape:", embeddings.shape)
print("First vector:", embeddings[0])


In [18]:
# 2. 엉망인 파일명 리스트 (Input Data)
filenames = [
    "Inj_Machine_Log_01.csv",       # 약어 사용
    "Plastic_Molding_Data.csv",     # 동의어 사용
    "Robot_Arm_Axis_X.csv",         # 구체적 부품
    "Auto_Welder_Final.csv",        # 용접기
    "Factory_Pump_Vib.csv",         # 펌프
    "M01_Unknown.csv",               # 의미 불명 (매핑 실패 예상)
    "CNC_Machine_Data.csv",
    "cnc.csv",
    "cnc_data.csv",
    "cnc_data_01.csv",
    "cnc_data_02.csv",
    "cnc_data_03.csv",
    "cnc_data_04.csv",
    
]

# 3. 온톨로지 표준 클래스 정의 (Target Classes)
# AI가 파일명을 보고 이 중 하나를 골라야 함
ontology_classes = [
    "Injection_Molding_Machine", # 사출기
    "Welding_Robot",             # 용접 로봇
    "Industrial_Pump",           # 펌프
    "CNC_Machine",               # CNC
    "Conveyor_Belt"              # 컨베이어
]

In [19]:
def preprocess_filename(fname):
    # 확장자 제거 및 특수문자를 공백으로 변환하여 '문장'처럼 만듦
    name = fname.replace('.csv', '').replace('_', ' ').replace('-', ' ')
    return name

In [20]:
import pandas as pd
def map_filenames_to_classes(files, classes):
    # 임베딩
    clean_names = [preprocess_filename(f) for f in files]
    embeddings_files = model.encode(clean_names)
    embeddings_classes = model.encode(classes)
    
    # 유사도 계산
    scores = util.cos_sim(embeddings_files, embeddings_classes)
    
    results = []
    for i, fname in enumerate(files):
        best_idx = scores[i].argmax().item()
        best_score = scores[i][best_idx].item()
        best_class = classes[best_idx]
        
        # 유사도가 낮으면(0.4 미만) 분류 보류
        category = best_class if best_score > 0.4 else "Unclassified"
        
        results.append({
            "Filename": fname,
            "Interpreted_As": clean_names[i],
            "Mapped_Class": category,
            "Confidence": round(best_score, 3)
        })
        
    return pd.DataFrame(results)

In [ ]:
import polars as pl
import os
from rdflib import Graph, Literal, RDF, Namespace, XSD
import urllib.parse
from datetime import datetime

# 1. 설정
INPUT_FOLDER = "./factory_data"  # 데이터가 저장된 폴더
META_FILE = "metadata_ontology.ttl"

# 네임스페이스 설정
BASE_URI = "http://factory.org/meta/"
META = Namespace(BASE_URI)
g = Graph()
g.bind("meta", META)

def build_metadata_dataset():
    print("🧐 메타 데이터셋 구축을 시작합니다...")
    
    # 폴더 내 파일 탐색
    if not os.path.exists(INPUT_FOLDER):
        os.makedirs(INPUT_FOLDER)
        print(f"   [안내] '{INPUT_FOLDER}' 폴더가 생성되었습니다. 데이터를 넣고 다시 실행해 주세요.")
        return

    files = [f for f in os.listdir(INPUT_FOLDER) if f.endswith(".csv")]
    
    for filename in files:
        file_path = os.path.join(INPUT_FOLDER, filename)
        
        # 1. [핵심] Polars Lazy API로 스키마만 빠르게 스캔 (데이터 로딩 X)
        try:
            # scan_csv는 실제 데이터를 메모리에 올리지 않습니다. 매우 빠릅니다.
            lf = pl.scan_csv(file_path)
            schema = lf.collect_schema() # 컬럼 정보만 가져옴
        except Exception as e:
            print(f"   [주의] 파일을 읽을 수 없습니다: {filename} ({e})")
            continue

        # 2. 데이터셋(파일) 객체 생성
        # 파일명을 ID로 사용
        dataset_name = os.path.splitext(filename)[0]
        dataset_uri = META[urllib.parse.quote(dataset_name)]
        
        g.add((dataset_uri, RDF.type, META.Dataset))
        g.add((dataset_uri, META.hasFilePath, Literal(os.path.abspath(file_path)))) # 물리적 경로 저장
        g.add((dataset_uri, META.hasFileName, Literal(filename)))
        
        # 3. 컬럼 정보(메타데이터) 등록
        print(f"   Process: {filename} (컬럼 {len(schema)}개 발견)")
        
        for col_name, data_type in schema.items():
            # 컬럼 객체 생성 (Dataset_ColumnName)
            col_id = f"{dataset_name}_{col_name}"
            col_uri = META[urllib.parse.quote(col_id)]
            
            # 관계 설정: 데이터셋 --[hasColumn]--> 컬럼
            g.add((col_uri, RDF.type, META.DataColumn))
            g.add((dataset_uri, META.hasColumn, col_uri))
            
            # 컬럼 속성 저장 (이름, 데이터 타입)
            g.add((col_uri, META.columnName, Literal(col_name)))
            g.add((col_uri, META.dataType, Literal(str(data_type))))

    # 결과 저장
    g.serialize(destination=META_FILE, format="turtle")
    print(f"✅ 메타 데이터셋 구축이 완료되었습니다! ({META_FILE})")

# 실행
if __name__ == "__main__":
    # 테스트용 더미 파일 생성 (없을 경우)
    if not os.path.exists(INPUT_FOLDER):
        os.makedirs(INPUT_FOLDER)
        with open(f"{INPUT_FOLDER}/Motor_Log_2024.csv", "w") as f:
            f.write("time,currR,currS,currT,Unknown_X\n10:00,10,10,10,0.1")
            
    build_metadata_dataset()

In [21]:
# 실행
df_result = map_filenames_to_classes(filenames, ontology_classes)
print(df_result)

                    Filename        Interpreted_As               Mapped_Class  \
0     Inj_Machine_Log_01.csv    Inj Machine Log 01               Unclassified   
1   Plastic_Molding_Data.csv  Plastic Molding Data  Injection_Molding_Machine   
2       Robot_Arm_Axis_X.csv      Robot Arm Axis X              Welding_Robot   
3      Auto_Welder_Final.csv     Auto Welder Final              Welding_Robot   
4       Factory_Pump_Vib.csv      Factory Pump Vib            Industrial_Pump   
5            M01_Unknown.csv           M01 Unknown               Unclassified   
6       CNC_Machine_Data.csv      CNC Machine Data                CNC_Machine   
7                    cnc.csv                   cnc                CNC_Machine   
8               cnc_data.csv              cnc data                CNC_Machine   
9            cnc_data_01.csv           cnc data 01                CNC_Machine   
10           cnc_data_02.csv           cnc data 02                CNC_Machine   
11           cnc_data_03.csv

In [22]:
df_result

,Filename,Interpreted_As,Mapped_Class,Confidence
0,Inj_Machine_Log_01.csv,Inj Machine Log 01,Unclassified,0.337
1,Plastic_Molding_Data.csv,Plastic Molding Data,Injection_Molding_Machine,0.437
2,Robot_Arm_Axis_X.csv,Robot Arm Axis X,Welding_Robot,0.406
3,Auto_Welder_Final.csv,Auto Welder Final,Welding_Robot,0.557
4,Factory_Pump_Vib.csv,Factory Pump Vib,Industrial_Pump,0.622
5,M01_Unknown.csv,M01 Unknown,Unclassified,0.233
6,CNC_Machine_Data.csv,CNC Machine Data,CNC_Machine,0.806
7,cnc.csv,cnc,CNC_Machine,0.756
8,cnc_data.csv,cnc data,CNC_Machine,0.713
9,cnc_data_01.csv,cnc data 01,CNC_Machine,0.663


In [23]:
from rdflib import Graph, RDF, Namespace, Literal

g = Graph()
FACT = Namespace("http://factory.org/")

for index, row in df_result.iterrows():
    if row['Mapped_Class'] == "Unclassified":
        continue
        
    # 1. 데이터셋 객체 생성 (파일명 기반)
    dataset_uri = FACT[row['Filename'].replace('.csv', '')]
    
    # 2. 매핑된 클래스 가져오기
    # 예: Injection_Molding_Machine 클래스
    class_uri = FACT[row['Mapped_Class']]
    
    # 3. [핵심] 데이터셋을 해당 클래스의 데이터라고 정의
    # 의미: "Inj_Machine_Log_01.csv는 사출기 데이터의 일종이다"
    g.add((dataset_uri, FACT.isDataOf, class_uri))
    
    # 추가: 신뢰도 점수도 메타데이터로 저장 (나중에 검증용)
    g.add((dataset_uri, FACT.mappingConfidence, Literal(row['Confidence'])))

In [25]:
g.serialize(destination='factory_data.ttl', format='turtle')

<Graph identifier=Nd1fe5ecce7be4316b3e2b38348211c4e (<class 'rdflib.graph.Graph'>)>

- mapped_machine -> mapped_col


In [28]:
df

NameError: name 'df' is not defined

In [38]:
import polars as pl
import pandas as pd
import os
import re
from sentence_transformers import SentenceTransformer, util
from rdflib import Graph, Literal, RDF, Namespace, XSD
import urllib.parse
from typing import Dict, List, Tuple, Optional
from dataclasses import dataclass

# ============================================
# 1. 규칙 기반 매핑 (Rule-based Mapping)
# ============================================
def build_hybrid_ontology(
    input_folder: str,
    ontology_classes: List[str],
    output_file: str = "metadata_ontology.ttl",
    model_name: str = 'sentence-transformers/all-MiniLM-L6-v2'
):
    """
    하이브리드 접근법으로 온톨로지 구축
    
    Args:
        input_folder: CSV 파일이 있는 폴더
        ontology_classes: 표준 온톨로지 클래스 리스트
        output_file: 출력 TTL 파일명
        model_name: SentenceTransformer 모델명
    """
    print("🧐 하이브리드 온톨로지 구축을 시작합니다...")
    
    # 네임스페이스 설정
    BASE_URI = "http://factory.org/meta/"
    META = Namespace(BASE_URI)
    FACT = Namespace("http://factory.org/")
    g = Graph()
    g.bind("meta", META)
    g.bind("fact", FACT)
    
    # 하이브리드 매퍼 초기화
    mapper = HybridMapper(ontology_classes, model_name)
    
    # 폴더 내 파일 탐색
    if not os.path.exists(input_folder):
        os.makedirs(input_folder)
        print(f"   [안내] '{input_folder}' 폴더가 생성되었습니다.")
        # 빈 DataFrame과 Graph 반환
        return pd.DataFrame(), g
    
    files = [f for f in os.listdir(input_folder) if f.endswith((".csv", ".CSV"))]
    
    if not files:
        print(f"   [경고] '{input_folder}' 폴더에 CSV 파일이 없습니다.")
        # 빈 DataFrame과 Graph 반환
        return pd.DataFrame(), g
    
    # 파일별 컬럼 정보 수집 (스키마만 스캔)
    file_columns = {}
    for filename in files:
        file_path = os.path.join(input_folder, filename)
        try:
            lf = pl.scan_csv(file_path)
            schema = lf.collect_schema()
            file_columns[filename] = list(schema.keys())
        except Exception as e:
            print(f"   [주의] 파일을 읽을 수 없습니다: {filename} ({e})")
            file_columns[filename] = []
    
    # 하이브리드 매핑 수행
    mapping_df = mapper.map_files(files, file_columns)
    
    print("\n📊 매핑 결과:")
    print(mapping_df.to_string(index=False))
    
    # 온톨로지 구축
    for _, row in mapping_df.iterrows():
        filename = row['Filename']
        mapped_class = row['Mapped_Class']
        confidence = row['Confidence']
        method = row['Method']
        
        if mapped_class == "Unclassified":
            print(f"   ⚠️  {filename}: 매핑 실패 (수동 검토 필요)")
            continue
        
        # 파일 경로
        file_path = os.path.join(input_folder, filename)
        dataset_name = os.path.splitext(filename)[0]
        dataset_uri = FACT[urllib.parse.quote(dataset_name)]
        class_uri = FACT[urllib.parse.quote(mapped_class)]
        
        # 데이터셋 객체 생성
        g.add((dataset_uri, RDF.type, META.Dataset))
        g.add((dataset_uri, META.hasFileName, Literal(filename)))
        g.add((dataset_uri, META.hasFilePath, Literal(os.path.abspath(file_path))))
        
        # 클래스 매핑
        g.add((dataset_uri, FACT.isDataOf, class_uri))
        g.add((dataset_uri, META.mappingConfidence, Literal(float(confidence), datatype=XSD.float)))
        g.add((dataset_uri, META.mappingMethod, Literal(method)))
        
        # 컬럼 정보 추가
        if filename in file_columns:
            for col_name in file_columns[filename]:
                col_id = f"{dataset_name}_{col_name}"
                col_uri = META[urllib.parse.quote(col_id)]
                
                g.add((col_uri, RDF.type, META.DataColumn))
                g.add((dataset_uri, META.hasColumn, col_uri))
                g.add((col_uri, META.columnName, Literal(col_name)))
        
        print(f"   ✅ {filename} → {mapped_class} ({method}, confidence: {confidence:.2f})")
    
    # 결과 저장
    if len(mapping_df) > 0:  # 파일이 있을 때만 저장
        g.serialize(destination=output_file, format="turtle")
        print(f"\n✅ 온톨로지 구축 완료! ({output_file})")
        
        # 통계 출력
        method_counts = mapping_df['Method'].value_counts()
        print("\n📈 매핑 방법 통계:")
        for method, count in method_counts.items():
            print(f"   {method}: {count}개")
    
    return mapping_df, g

In [40]:
if __name__ == "__main__":
    # 온톨로지 클래스 정의
    ontology_classes = [
        "Injection_Molding_Machine",
        "Welding_Robot",
        "Industrial_Pump",
        "CNC_Machine",
        "Conveyor_Belt",
        "Motor"
    ]
    
    # 온톨로지 구축 실행
    mapping_df, graph = build_hybrid_ontology(
        input_folder="./csv_data",
        ontology_classes=ontology_classes,
        output_file="metadata_ontology_hybrid.ttl"
    )
    
    # 결과 확인
    print("\n" + "="*50)
    print("매핑 결과 상세:")
    print("="*50)
    print(mapping_df)

🧐 하이브리드 온톨로지 구축을 시작합니다...

📊 매핑 결과:
   Filename Interpreted_As Mapped_Class  Confidence           Method                              Column_Hint
deoksan.csv        deoksan        Motor         0.8 column_inference Detected from columns: time, curr, currR
   ✅ deoksan.csv → Motor (column_inference, confidence: 0.80)

✅ 온톨로지 구축 완료! (metadata_ontology_hybrid.ttl)

📈 매핑 방법 통계:
   column_inference: 1개

매핑 결과 상세:
      Filename Interpreted_As Mapped_Class  Confidence            Method  \
0  deoksan.csv        deoksan        Motor         0.8  column_inference   

                                Column_Hint  
0  Detected from columns: time, curr, currR  
